# Accuracy Measurements for Object Detection and Tracking with YOLOv8, ByteTrack, and the VIRAT Dataset

## 1. Data Import
## 2.

## 1. Data Import
- obtain sample video from dataset for inference

In [47]:
import pandas as pd
import torch

In [1]:
# upload sample file: 'VIRAT_S_000200_03_000657_000899_first300.mp4'
# this is the first 300 frames of 'VIRAT_S_000200_03_000657_000899.mp4'

!gdown "https://drive.google.com/uc?id=1C-HPOEIdZnJm_RSbYGXH6gMLhlmobaxn" -O test.mp4

Downloading...
From: https://drive.google.com/uc?id=1C-HPOEIdZnJm_RSbYGXH6gMLhlmobaxn
To: /content/test.mp4
100% 10.4M/10.4M [00:00<00:00, 52.9MB/s]


I used `ffmpeg -i VIRAT_S_000200_03_000657_000899.mp4 -c:v copy -frames:v 300 -r 30 -an ~/Desktop/VIRAT_S_000200_03_000657_000899_first300.mp4     
`

In [2]:
# display details of clip
!ffprobe -v quiet -print_format json -show_format -show_streams test.mp4

{
    "streams": [
        {
            "index": 0,
            "codec_name": "mpeg4",
            "codec_long_name": "MPEG-4 part 2",
            "profile": "Simple Profile",
            "codec_type": "video",
            "codec_time_base": "1/30",
            "codec_tag_string": "mp4v",
            "codec_tag": "0x7634706d",
            "width": 1280,
            "height": 720,
            "coded_width": 1280,
            "coded_height": 720,
            "has_b_frames": 0,
            "sample_aspect_ratio": "1:1",
            "display_aspect_ratio": "16:9",
            "pix_fmt": "yuv420p",
            "level": 3,
            "color_range": "tv",
            "color_space": "bt709",
            "color_transfer": "bt709",
            "color_primaries": "bt709",
            "chroma_location": "left",
            "refs": 1,
            "quarter_sample": "false",
            "divx_packed": "false",
            "r_frame_rate": "30/1",
            "avg_frame_rate": "30/1",
            "tim

In [3]:
!ls -lh

total 10M
drwxr-xr-x 1 root root 4.0K Jun 23 01:15 sample_data
-rw-r--r-- 1 root root  10M Jun 23 18:46 test.mp4


## 2. Ground Truth
- obtain the ground truth bounding boxes data
- store in Pandas DataFrame 'gtdf'

In [4]:
# VIRAT_S_000200_03_000657_000899.types.yml
!gdown "https://drive.google.com/uc?id=12h_35hXzoSciduBzDWmpKB2-rgUuraiN" -O test.types.yml

Downloading...
From: https://drive.google.com/uc?id=12h_35hXzoSciduBzDWmpKB2-rgUuraiN
To: /content/test.types.yml
100% 2.71k/2.71k [00:00<00:00, 19.3MB/s]


In [5]:
# VIRAT_S_000200_03_000657_000899.regions.yml
!gdown "https://drive.google.com/uc?id=1Ieau47ZxLLpE6mw04XwjfANeQPB9m7hY" -O test.regions.yml

Downloading...
From: https://drive.google.com/uc?id=1Ieau47ZxLLpE6mw04XwjfANeQPB9m7hY
To: /content/test.regions.yml
100% 12.6M/12.6M [00:00<00:00, 142MB/s]


In [6]:
# VIRAT_S_000200_03_000657_000899.geom.yml
!gdown "https://drive.google.com/uc?id=1UH9s2MPSZFdJJ7TD827DrmqGPtR_CWdQ" -O test.geom.yml

Downloading...
From: https://drive.google.com/uc?id=1UH9s2MPSZFdJJ7TD827DrmqGPtR_CWdQ
To: /content/test.geom.yml
100% 8.54M/8.54M [00:00<00:00, 84.5MB/s]


In [7]:
# VIRAT_S_000200_03_000657_000899.activities.yml
!gdown "https://drive.google.com/uc?id=1tTAnWLE5f9FhbWCe4vElR7gD9aK0jYin" -O test.activities.yml

Downloading...
From: https://drive.google.com/uc?id=1tTAnWLE5f9FhbWCe4vElR7gD9aK0jYin
To: /content/test.activities.yml
100% 14.3k/14.3k [00:00<00:00, 61.0MB/s]


In [8]:
!ls -lh

total 31M
drwxr-xr-x 1 root root 4.0K Jun 23 01:15 sample_data
-rw-r--r-- 1 root root  14K Jun 23 18:46 test.activities.yml
-rw-r--r-- 1 root root 8.2M Jun 23 18:46 test.geom.yml
-rw-r--r-- 1 root root  10M Jun 23 18:46 test.mp4
-rw-r--r-- 1 root root  12M Jun 23 18:46 test.regions.yml
-rw-r--r-- 1 root root 2.7K Jun 23 18:46 test.types.yml


In [9]:
# get bounding boxes, classes and track ids for detections in the first frame
import yaml

file_path = 'test.geom.yml'
with open(file_path, 'r') as file:
    geom = yaml.safe_load(file)
file_path = 'test.types.yml'
with open(file_path, 'r') as file:
    types = yaml.safe_load(file)

### Create Ground Truth DataFrame

AS pd.DataFrame </br>
format: </br>
{idx: geom_id0, </br>
track_id: geom_id1, </br>
label: types_cset3-key_on_id1, </br>
conf: types_cset3-value_on_id1, </br>
frame: geom_ts0, </br>
xmin: geom_g0.split(' ')[0], ymin: geom_g0.split(' ')[1], xmax: geom_g0.split(' ')[2], ymax: geom_g0.split(' ')[3]}

In [77]:
# helper function to get gt labels and confidence from types

def get_labels_conf(types, frame_num):
  # returns lists: labels, conf
  labels = []
  conf = []
  for i in types:
    try:
      if (i['types']['id1'] in track_id):
          label, conf = next(iter(i['types']['cset3'].items()))
          labels.append(label)
          conf.appens(conf)
    except:
      pass

  return labels, conf


# helper function to get labels as integers. Only working for 'Person', 'Bike' and 'Vehicle' currently [0,1,2]
def labels_as_int(label):
  labels = []
  for i in range(len(label)):
    if label[i] == 'Person':
      labels.append(0)
    elif label[i] == 'Bike':
      labels.append(1)
    elif label[i] == 'Vehicle':
      labels.append(2)
    else:
      labels.append(-1)
  return labels

In [78]:
# create a ground truth dataframe, gtdf

detections = []
frame_num = 0

# sample of geom with detections by frame_num
for i in geom:
  try:
    if (i['geom']['ts0'] == frame_num):
      detections.append(i)
  except:
    pass

# parse data from detections
idx = [i['geom']['id0'] for i in detections]
track_id = [i['geom']['id1'] for i in detections]
label, conf = get_labels_conf(types, frame_num)
frame = [i['geom']['ts0'] for i in detections]
xmin, ymin, xmax, ymax = [], [], [], []
for i in detections:
  bb = i['geom']['g0'].split(' ')
  xmin.append(int(bb[0]))
  ymin.append(int(bb[1]))
  xmax.append(int(bb[2]))
  ymax.append(int(bb[3]))

# get class labels as int for comparison with predictions
label_int = labels_as_int(label)

gt = {'idx_gt': idx, 'track_id_gt': track_id, 'label_gt': label,
      'label_as_int_gt': label_int, 'conf_gt': conf, 'frame_gt': frame,
      'xmin_gt': xmin, 'ymin_gt': ymin, 'xmax_gt': xmax, 'ymax_gt': ymax}
gtdf = pd.DataFrame(gt)
gtdf

,idx_gt,track_id_gt,label_gt,label_as_int_gt,conf_gt,frame_gt,xmin_gt,ymin_gt,xmax_gt,ymax_gt
0,0,1,Vehicle,2,1.0,0,946,327,1010,360
1,7243,2,Vehicle,2,1.0,0,648,495,801,559
2,14486,3,Vehicle,2,1.0,0,114,375,237,419
3,21729,4,Vehicle,2,1.0,0,102,349,212,386
4,28972,5,Vehicle,2,1.0,0,902,659,1023,718
5,34911,33,Person,0,1.0,0,281,517,320,577
6,39594,34,Person,0,1.0,0,730,392,756,437
7,46239,35,Person,0,1.0,0,1149,644,1185,714
8,51025,5000,Parking_Meter,-1,1.0,0,1056,368,1070,402
9,58268,5001,Dumpster,-1,1.0,0,69,291,117,328


In [79]:
gtdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   idx_gt           12 non-null     int64  
 1   track_id_gt      12 non-null     int64  
 2   label_gt         12 non-null     object 
 3   label_as_int_gt  12 non-null     int64  
 4   conf_gt          12 non-null     float64
 5   frame_gt         12 non-null     int64  
 6   xmin_gt          12 non-null     int64  
 7   ymin_gt          12 non-null     int64  
 8   xmax_gt          12 non-null     int64  
 9   ymax_gt          12 non-null     int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 1.1+ KB


## 3. Get Predictions
- get inference from YOLOv8 with ByteTracker on this test clip
- store predictions in Pandas DataFrame 'preddf'

Install YOLOv8

In [28]:
!pip -q install ultralytics
!pip -q install lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [29]:
from ultralytics import YOLO
import lap

model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.7MB/s]


In [30]:
results = model.track(source='/content/test.mp4', conf=0.12, iou=0.5,
                      device=0, save_txt=True, imgsz=1280, classes=[0,1,2],
                      tracker="bytetrack.yaml", stream=True)
list_results = list(results)


video 1/1 (1/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 71.6ms
video 1/1 (2/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (3/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (4/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (5/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (6/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (7/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (8/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 14.7ms
video 1/1 (9/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 40.6ms
video 1/1 (10/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 37.9ms
video 1/1 (11/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (12/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 13.4ms
video 1/1 (13/300) /content/test.mp4: 736x1280 3 persons, 5 cars, 15.1ms
video 1/1 (14/300) /content/test.mp4: 736x1280 3 persons, 5

In [36]:
# frame 1 detections as .txt file(s)
!cat runs/detect/track/labels/test_1.txt

2 0.137419 0.553092 0.0945308 0.0548357 1
2 0.763876 0.478838 0.0469688 0.0366087 2
2 0.123751 0.511463 0.0826546 0.044233 3
2 0.567661 0.733765 0.111929 0.0792986 4
0 0.232489 0.761387 0.019395 0.080564 5
0 0.581185 0.575837 0.0138645 0.05707 6
2 0.751597 0.962718 0.090546 0.0741219 7
0 0.909582 0.944357 0.0208885 0.095949 8


.txt files format ^
- [class, xmid, ymid, w, h, track_id]</br></br>

In [37]:
# frame 1 detection as tensor objects
list_results[0].boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[1.1540e+02, 3.7849e+02, 2.3640e+02, 4.1797e+02, 1.0000e+00, 8.8784e-01, 2.0000e+00],
        [9.4770e+02, 3.3158e+02, 1.0078e+03, 3.5794e+02, 2.0000e+00, 8.5510e-01, 2.0000e+00],
        [1.0550e+02, 3.5233e+02, 2.1130e+02, 3.8418e+02, 3.0000e+00, 8.5349e-01, 2.0000e+00],
        [6.5497e+02, 4.9976e+02, 7.9824e+02, 5.5686e+02, 4.0000e+00, 8.5208e-01, 2.0000e+00],
        [2.8517e+02, 5.1920e+02, 3.1000e+02, 5.7720e+02, 5.0000e+00, 8.3465e-01, 0.0000e+00],
        [7.3504e+02, 3.9406e+02, 7.5279e+02, 4.3515e+02, 6.0000e+00, 7.6037e-01, 0.0000e+00],
        [9.0410e+02, 6.6647e+02, 1.0200e+03, 7.1984e+02, 7.0000e+00, 7.5590e-01, 2.0000e+00],
        [1.1509e+03, 6.4540e+02, 1.1776e+03, 7.1448e+02, 8.0000e+00, 6.6577e-01, 0.0000e+00]])
cls: tensor([2., 2., 2., 2., 0., 0., 2., 0.])
conf: tensor([0.8878, 0.8551, 0.8535, 0.8521, 0.8347, 0.7604, 0.7559, 0.6658])
data: tensor([[1.1540e+02, 3.7849e+02, 2.3640e+02, 4

ultralytics.yolo.engine.results.Boxes attribute formats ^:
- Boxes.boxes, Boxes.data use [xmin, ymin, xmax, ymax, track_id, conf, class]
- xywh, xywhn use [xmid, ymid, w, h]
- xyxy, xyxyn use [xmin, ymin, xmax, ymax]

In [38]:
list_results[0].boxes.data[0][6]

tensor(2.)

### Create DataFrame for predictions


In [50]:
# Create dictionary of predictions

frame_num = 0

track_id = []
label = []
conf = []
frame = []
xmin, ymin, xmax, ymax = [], [], [], []

for i in range(len(list_results[frame_num].boxes.data)):
  bb = [torch.round(j) for j in iter(list_results[frame_num].boxes.data[i])]
  track_id.append(int(bb[4]))
  label.append(int(bb[6]))
  conf.append(int(bb[5]))
  frame.append(frame_num)
  xmin.append(int(bb[0]))
  ymin.append(int(bb[1]))
  xmax.append(int(bb[2]))
  ymax.append(int(bb[3]))

pred = {'track_id': track_id, 'label': label, 'conf': conf, 'frame': frame, 'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax}

preddf = pd.DataFrame(pred)

In [51]:
preddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   track_id  8 non-null      int64
 1   label     8 non-null      int64
 2   conf      8 non-null      int64
 3   frame     8 non-null      int64
 4   xmin      8 non-null      int64
 5   ymin      8 non-null      int64
 6   xmax      8 non-null      int64
 7   ymax      8 non-null      int64
dtypes: int64(8)
memory usage: 640.0 bytes


In [52]:
preddf

,track_id,label,conf,frame,xmin,ymin,xmax,ymax
0,1,2,1,0,115,378,236,418
1,2,2,1,0,948,332,1008,358
2,3,2,1,0,106,352,211,384
3,4,2,1,0,655,500,798,557
4,5,0,1,0,285,519,310,577
5,6,0,1,0,735,394,753,435
6,7,2,1,0,904,666,1020,720
7,8,0,1,0,1151,645,1178,714


## 4. Calculate IOU

In [80]:
import numpy as np

def get_iou(boxA, boxB):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters:
    boxA -- [xmin, ymin, xmax, ymax] of box A
    boxB -- [xmin, ymin, xmax, ymax] of box B

    Returns:
    iou   -- Intersection over Union
    """
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection area and dividing it by the sum of prediction + ground-truth areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [81]:
# Create a cross join dataframe
gtdf['key'] = 1
preddf['key'] = 1
df_cross = pd.merge(gtdf, preddf, on ='key').drop("key", axis=1)

In [82]:
# Calculate IOU for each pair
df_cross['iou'] = df_cross.apply(lambda row: get_iou(row[['xmin_gt', 'ymin_gt', 'xmax_gt', 'ymax_gt']], row[['xmin', 'ymin', 'xmax', 'ymax']]), axis=1)

# Find the pairs with maximum IOU for each ground truth bounding box
df_matched = df_cross.loc[df_cross.groupby(['idx_gt'])['iou'].idxmax()]

# Drop any unmatched predictions
df_matched = df_matched[df_matched['iou'] > 0]
df_matched

,idx_gt,track_id_gt,label_gt,label_as_int_gt,conf_gt,frame_gt,xmin_gt,ymin_gt,xmax_gt,ymax_gt,track_id,label,conf,frame,xmin,ymin,xmax,ymax,iou
1,0,1,Vehicle,2,1.0,0,946,327,1010,360,2,2,1,0,948,332,1008,358,0.745249
11,7243,2,Vehicle,2,1.0,0,648,495,801,559,4,2,1,0,655,500,798,557,0.834366
16,14486,3,Vehicle,2,1.0,0,114,375,237,419,1,2,1,0,115,378,236,418,0.896416
26,21729,4,Vehicle,2,1.0,0,102,349,212,386,3,2,1,0,106,352,211,384,0.829303
38,28972,5,Vehicle,2,1.0,0,902,659,1023,718,7,2,1,0,904,666,1020,720,0.820890
44,34911,33,Person,0,1.0,0,281,517,320,577,5,0,1,0,285,519,310,577,0.628689
53,39594,34,Person,0,1.0,0,730,392,756,437,6,0,1,0,735,394,753,435,0.642512
63,46239,35,Person,0,1.0,0,1149,644,1185,714,8,0,1,0,1151,645,1178,714,0.746098
